In [74]:
import os

import pandas as pd

### Data preproc

In [75]:
df_train = pd.read_csv(
    os.path.join("data", "train.csv")
)

In [76]:
df_attrs = pd.read_csv(
    os.path.join("data", "attr.csv")
)

In [77]:
df_attrs.loc[:, "node_idx_global"] = df_attrs.index

In [78]:
df_train

,ego_id,u,v,t,x1,x2,x3
0,0,131,84,148.0,5.669200e-07,0.000000,0.0
1,0,135,164,396.7,6.246274e-02,0.000000,0.0
2,0,47,15,NaN,0.000000e+00,0.000000,1.0
3,0,5,4,594.5,4.962974e-02,0.000000,0.0
4,0,176,219,45.5,1.237935e+00,0.000000,0.0
...,...,...,...,...,...,...,...
122280367,1709396984692,3,5,34.6,2.307750e+00,1.098612,0.0
122280368,1709396984692,1,5,53.8,3.729143e+00,3.496508,1.0
122280369,1709396984692,1,7,1.5,4.286984e+00,0.000000,0.0
122280370,1709396984692,5,11,2.0,3.500757e+00,0.000000,0.0


In [79]:
df_attrs

,ego_id,u,age,city_id,sex,school,university,node_idx_global
0,0,227,68,-1,1,778293348,-1,0
1,0,45,38,237065842,1,82803468,238500268,1
2,0,142,60,237065842,1,196560139,-1,2
3,0,280,66,-1,2,963209731,720783270,3
4,0,41,18,-1,2,308862409,-1,4
...,...,...,...,...,...,...,...,...
14930743,1709396984692,2,16,492149712,2,769209871,-1,14930743
14930744,1709396984692,12,15,-1,1,-1,-1,14930744
14930745,1709396984692,18,23,-1,1,-1,-1,14930745
14930746,1709396984692,4,16,650683235,1,-1,-1,14930746


In [80]:
df_attrs.rename(
    columns={
        "u": "node_idx"
    },
    inplace=True,
)

In [81]:
df_train = df_train.merge(df_attrs, left_on=["ego_id", "u"], right_on=["ego_id", "node_idx"], how="left")

In [82]:
len(set(df_train["ego_id"]))

61786

In [83]:
ego_ids = list(df_train["ego_id"].unique())
split = 10000
train_ids = ego_ids[:split]
# val_ids = ego_ids[split:]

df_train_subset = df_train.loc[
    df_train.loc[:, "ego_id"].isin(train_ids)
]

In [84]:
df_train = df_train_subset.merge(df_attrs, left_on=["ego_id", "v"], right_on=["ego_id", "node_idx"], how="inner", suffixes=("_u", "_v"))

In [85]:
df_train.drop(columns=["node_idx_u", "node_idx_v"], inplace=True)

In [86]:
# df_train.loc[:, "x3"].nunique()
# df_train.loc[:, "x3"].sort_values()

In [87]:
target = df_train['x1']

In [88]:
drop_features = [
    'ego_id', 
    'u', 
    'v', 
    'x1', 
    'node_idx_global_u', 
    'node_idx_global_v'
]

In [89]:
cat_features = [
    'city_id_u',
    'city_id_v',
    'x3',
    'sex_u',
    'sex_v',
    'school_u',
    'school_v',
    'university_u',
    'university_v'
]

In [90]:
df_train.loc[:, "city_id_u"] = df_train.loc[:, "city_id_u"].fillna(0)
df_train.loc[:, "city_id_v"] = df_train.loc[:, "city_id_v"].fillna(0)
df_train.loc[:, "sex_u"] = df_train.loc[:, "sex_u"].fillna(0)
df_train.loc[:, "sex_v"] = df_train.loc[:, "sex_v"].fillna(0)
df_train.loc[:, "school_u"] = df_train.loc[:, "school_u"].fillna(0)
df_train.loc[:, "school_v"] = df_train.loc[:, "school_v"].fillna(0)
df_train.loc[:, "university_u"] = df_train.loc[:, "university_u"].fillna(0)
df_train.loc[:, "university_v"] = df_train.loc[:, "university_v"].fillna(0)

In [91]:
df_train[cat_features]

,city_id_u,city_id_v,x3,sex_u,sex_v,school_u,school_v,university_u,university_v
0,237065842.0,237065842,0.0,1.0,1,-1.0,182400947,-1.0,310894832
1,237065842.0,237065842,0.0,1.0,1,625119395.0,182400947,815498626.0,310894832
2,237065842.0,237065842,0.0,1.0,1,-1.0,182400947,649586426.0,310894832
3,237065842.0,237065842,0.0,1.0,1,657513194.0,182400947,188886628.0,310894832
4,237065842.0,237065842,0.0,1.0,1,682233918.0,182400947,532275706.0,310894832
...,...,...,...,...,...,...,...,...,...
18554525,583492759.0,583492759,0.0,1.0,1,156381099.0,918317932,-1.0,502032137
18554526,583492759.0,583492759,0.0,1.0,1,147516338.0,53304186,392811042.0,772479732
18554527,583492759.0,583492759,0.0,2.0,1,17030511.0,17030511,-1.0,192890972
18554528,583492759.0,583492759,0.0,1.0,1,147516338.0,-1,392811042.0,-1


In [92]:
# df_train.info()

In [93]:
df_train[cat_features] = df_train[cat_features].astype(int)

In [94]:
df_train[df_train.loc[:,"ego_id"] == 8]

,ego_id,u,v,t,x1,x2,x3,age_u,city_id_u,sex_u,school_u,university_u,node_idx_global_u,age_v,city_id_v,sex_v,school_v,university_v,node_idx_global_v


In [95]:
df_train.drop(drop_features, axis=1, inplace=True)

### Model

In [96]:
from catboost import CatBoostRegressor, Pool

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [97]:
'''Разбиение на тест и трейн 70/30'''
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(df_train, target, test_size=0.3, random_state=123)

In [98]:
cat_features_index = [i for i, item in enumerate(df_train.columns) if item in cat_features]

In [99]:
'''Catboost'''

cb = CatBoostRegressor(cat_features = cat_features_index,  task_type = 'GPU', loss_function='RMSE')

%time
cb.fit(X_train_split, y_train_split)

CPU times: total: 0 ns
Wall time: 0 ns


: 

: 

In [ ]:
mean_squared_error(y_test_split, cb.predict(X_test_split), squared=False)

### Submission

#### Data preproc

In [ ]:
df_submission = pd.read_csv(
    os.path.join("data", "submission.csv")
)

In [ ]:
df_test = pd.read_csv(
    os.path.join("data", "test.csv")
)

In [ ]:
# df_submission[df_submission.loc[:,"ego_id"] == 8].sort_values(by=["u","v"])

In [ ]:
df_submission_enriched = df_test.merge(df_submission, on=["ego_id", "u", "v"], how="inner", copy=False, suffixes=["_test", "_submission"])

In [ ]:
df_submission_enriched.drop(columns=["x1_test", "x1_submission"], inplace=True)

In [ ]:
df_submission_enriched.drop_duplicates(subset=["ego_id", "u", "v"], inplace=True)

In [ ]:
df_submission_enriched

In [ ]:
df_submission_enriched

In [ ]:
df_submission_enriched = df_submission_enriched.merge(df_attrs, left_on=["ego_id", "u"], right_on=["ego_id", "node_idx"], how="left")

In [ ]:
df_submission_enriched = df_submission_enriched.merge(df_attrs, left_on=["ego_id", "v"], right_on=["ego_id", "node_idx"], how="left", suffixes=("_u", "_v"))

In [ ]:
df_submission_enriched.drop(columns=["node_idx_u", "node_idx_v"], inplace=True)

In [ ]:
df_submission_enriched.loc[:, "city_id_v"] = df_submission_enriched.loc[:, "city_id_v"].fillna(0)
df_submission_enriched.loc[:, "city_id_u"] = df_submission_enriched.loc[:, "city_id_u"].fillna(0)
df_submission_enriched.loc[:, "sex_u"] = df_submission_enriched.loc[:, "sex_u"].fillna(0)
df_submission_enriched.loc[:, "sex_v"] = df_submission_enriched.loc[:, "sex_v"].fillna(0)
df_submission_enriched.loc[:, "school_u"] = df_submission_enriched.loc[:, "school_u"].fillna(0)
df_submission_enriched.loc[:, "school_v"] = df_submission_enriched.loc[:, "school_v"].fillna(0)
df_submission_enriched.loc[:, "university_u"] = df_submission_enriched.loc[:, "university_u"].fillna(0)
df_submission_enriched.loc[:, "university_v"] = df_submission_enriched.loc[:, "university_v"].fillna(0)

In [ ]:
df_submission_enriched[cat_features]

In [ ]:
# df_train.info()

In [ ]:
df_submission_enriched[cat_features] = df_submission_enriched[cat_features].astype(int)

In [ ]:
df_submission_enriched

In [ ]:
df_submission

In [ ]:
df_submission_edges = df_submission_enriched.loc[:, ["ego_id", "u", "v"]]
df_submission_edges

In [ ]:
drop_features_submission = [
    'ego_id', 
    'u', 
    'v', 
    'node_idx_global_u', 
    'node_idx_global_v'
]

In [ ]:
df_submission_enriched.drop(drop_features_submission, axis=1, inplace=True)

In [ ]:
df_submission_enriched

In [ ]:
df_submission_enriched

In [ ]:
df_submission_edges["x1"] = cb.predict(df_submission_enriched)

In [ ]:
df_submission_edges

In [ ]:
df_submission_edges.sort_values(["ego_id", "u", "v"], inplace=True)

In [ ]:
df_submission_edges.nunique()

In [ ]:
df_submission_edges

In [ ]:
df_submission_edges.to_csv("submits/submission_catboost_2.csv", index=False)

In [ ]:
df_submission_edges.sort_values

In [ ]:
df_submission_edges.dtypes

In [ ]:
a = 1.606742e+00

In [ ]:
type(a)

In [ ]:
b = 0.35925092226941047
type(b)